# 匹配滤波算法及其变体

必要的库和依赖

In [1]:
import numpy as np
import sys
import os
sys.path.append("C:\\Users\\RS\\VSCode\\matchedfiltermethod")
import MyFunctions.AHSI_data as ad
import MyFunctions.EMIT_data as ed
from MyFunctions.needed_function import open_unit_absorption_spectrum

## 整幅影像进行计算

### 常规匹配滤波算法

In [ ]:

# original matched filter algorithm 整幅图像进行计算
def matched_filter(data_cube: np.array, unit_absorption_spectrum: np.array, albedoadjust, iterate, sparsity):        
    """
    Calculate the methane enhancement of the image data based on the original matched filter
    and the unit absorption spectrum.
    
    :param data_array: numpy array of the image data
    :param unit_absorption_spectrum: list of the unit absorption spectrum
    """
    # 获取波段 行数 列数 初始化 concentration 数组，大小与卫星数据尺寸一直
    bands, rows, cols = data_cube.shape
    concentration = np.zeros((rows, cols))
    
    # 对于非空列，取均值作为背景光谱，再乘以单位吸收光谱，得到目标光谱
    background_spectrum = np.nanmean(data_cube, axis=(1,2))
    target_spectrum = background_spectrum*unit_absorption_spectrum
    radiancediff_with_bg =data_cube - background_spectrum[:, None,None]
    
    # 计算协方差矩阵，并获得其逆矩阵
    d_covariance = radiancediff_with_bg
    covariance = np.zeros((bands, bands))
    for row in range(rows): 
        for col in range(cols):
            covariance += np.outer(d_covariance[:, row, col], d_covariance[:, row, col])
    covariance = covariance/(rows*cols)
    covariance_inverse = np.linalg.inv(covariance)
    
    # 判断是否进行反照率校正，若是，则通过背景光谱和实际光谱计算反照率校正因子
    albedo = np.ones((rows, cols))
    if albedoadjust:
        for row in range(rows):
            for col in range(cols):
                albedo[row, col] = (
                        (data_cube[:,row,col].T @ background_spectrum) /
                        (background_spectrum.T @ background_spectrum)
                )
    
    # 基于最优化公式计算每个像素的甲烷浓度增强值
    for row in range(rows):
        for col in range(cols):
            numerator = (radiancediff_with_bg[:,row,col].T @ covariance_inverse @ target_spectrum)
            denominator = albedo[row,col]*(target_spectrum.T @ covariance_inverse @ target_spectrum)
            concentration[row,col] = numerator/denominator
    
    # for row in range(rows):
    #     for col in range(cols):
    #         numerator = (radiancediff_with_bg[:,row,col].T @ target_spectrum)
    #         denominator = albedo[row,col]*(target_spectrum.T @ target_spectrum)
    #         concentration[row,col] = numerator/denominator
    
    # 判断是否进行迭代，若是，则进行如下迭代计算
    if iterate:
        # 初始化 l1filter 数组
        l1filter = np.zeros((rows,cols))
        for iter_num in range(5):
            print("iteration: No.", iter_num + 1)
            # 判断是否进行 l1filter 校正，若是则给予前一步的浓度值计算 l1filter
            if sparsity:
                for row in rows:
                    for col in cols:
                        l1filter = 1 / (concentration[row,col] + np.finfo(np.float64).tiny)
            
            # 更新背景光谱和目标光谱,并计算进行观测光谱和背景光谱差值   
            updated_array = data_cube - (albedo*concentration)[None,:,:]*target_spectrum[:,None,None]
            background_spectrum = np.mean(updated_array, axis=(1,2))
            target_spectrum = np.multiply(background_spectrum, unit_absorption_spectrum)
            radiancediff_with_bg = data_cube - background_spectrum[:,None,None]
            
            # 基于新的目标谱和背景光谱 计算协方差矩阵
            d_covariance = data_cube -(albedo*concentration)[None,:,:]*target_spectrum[:,None,None] - background_spectrum[:,None,None]
            covariance = np.zeros((bands, bands))
            for row in range(rows):
                for col in range(cols):
                    covariance += np.outer(d_covariance[:,row,col], d_covariance[:,row,col])
            covariance = covariance/(rows*cols)
            covariance_inverse = np.linalg.inv(covariance)

            # 基于最优化估计公式 计算新的甲烷浓度增强值
            for row in range(rows):
                for col in range(cols):
                    numerator = (radiancediff_with_bg[:,row,col].T @ covariance_inverse @ target_spectrum) - l1filter[row,col]
                    denominator = albedo[row,col] * (target_spectrum.T @ covariance_inverse @ target_spectrum)
                    concentration[row,col] = np.maximum(numerator / denominator, 0.0)
                  
            # for row in range(rows):
            #     for col in range(cols):
            #         numerator = (radiancediff_with_bg[:,row,col].T @ target_spectrum) - l1filter[row,col]
            #         denominator = albedo[row,col] * (target_spectrum.T @ target_spectrum)
            #         concentration[row,col] = np.maximum(numerator / denominator, 0.0)
    return concentration
 


### 多层匹配滤波算法

In [ ]:

# modified matched filter algorithm 整幅图像进行计算
def modified_matched_filter_copy(data_cube: np.array, unit_absorption_spectrum: np.array) -> np.array:
    """
    Calculate the methane enhancement of the image data based on the original matched filter
    and the unit absorption spectrum.

    :param data_cube: numpy array of the image data
    :param unit_absorption_spectrum: list of the unit absorption spectrum
    :param is_iterate: flag to decide whether to iterate the matched filter
    :param is_albedo: flag to decide whether to do the albedo correction
    :param is_filter: flag to decide whether to add the l1-filter correction
    :return: numpy array of methane enhancement result
    """
    # 获取 以 波段 行数 列数 为顺序的数据
    bands, rows, cols = data_cube.shape
    # 初始化 concentration 数组，大小与卫星数据尺寸一直
    concentration = np.zeros((rows, cols))
    # 对于非空列，取均值作为背景光谱，再乘以单位吸收光谱，得到目标光谱
    background_spectrum = np.nanmean(data_cube, axis=(1,2))
    target_spectrum = background_spectrum*unit_absorption_spectrum[0]
    radiancediff_with_bg = data_cube - background_spectrum[:, None,None]

    d_covariance = data_cube - background_spectrum[:, None,None]
    covariance = np.zeros((bands, bands))
    for row in range(rows): 
        for col in range(cols):
            covariance += np.outer(d_covariance[:, row, col], d_covariance[:, row, col])
    covariance = covariance/(rows*cols)
    covariance_inverse = np.linalg.inv(covariance)

    for row in range(rows):
        for col in range(cols):
            # 基于最优化公式计算每个像素的甲烷浓度增强值
            numerator = (radiancediff_with_bg[:,row,col].T @ covariance_inverse @ target_spectrum)
            denominator = (target_spectrum.T @ covariance_inverse @ target_spectrum)
            concentration[row,col] = numerator/denominator

    original_concentration = concentration.copy()
    levelon = True
    while levelon:
        adaptive_threshold = 4000
        i = 1
        # _, new_unit_absorption_spectrum = lookup_uas_interpolated(adaptive_threshold)
        new_unit_absorption_spectrum = unit_absorption_spectrum[i]
        high_concentration_mask = concentration > adaptive_threshold
        low_concentration_mask = concentration <= adaptive_threshold
        if np.sum(high_concentration_mask) > 0:
            # 注意：target_spectrum 的更新应该基于更准确的估计值
            background_spectrum = np.nanmean(data_cube - concentration * target_spectrum[:, None, None], axis=(1, 2))
            target_spectrum = background_spectrum * unit_absorption_spectrum
            new_background_spectrum = background_spectrum + adaptive_threshold * new_unit_absorption_spectrum
            high_target_spectrum = new_background_spectrum * new_unit_absorption_spectrum

            radiancediff_with_bg[:,low_concentration_mask] = (
                data_cube[:,low_concentration_mask] - background_spectrum[:,None]
            )
            radiancediff_with_bg[:,high_concentration_mask] = (
                data_cube[:,high_concentration_mask] - new_background_spectrum[:,None]
            )

            d_covariance[:,high_concentration_mask] = data_cube[:,high_concentration_mask] - (
                (concentration[high_concentration_mask]-adaptive_threshold)*high_target_spectrum[:,None] + new_background_spectrum[:,None]
            )
            d_covariance[:,low_concentration_mask] = data_cube[:,low_concentration_mask] - (
                concentration[low_concentration_mask]*target_spectrum[:,None] + background_spectrum[:,None]
            )
        
            covariance = np.zeros((bands, bands))
            for row in range(rows):
                for col in range(cols):
                    covariance += np.outer(d_covariance[:, row, col], d_covariance[:, row, col])
            covariance /= rows*cols
            covariance_inverse = np.linalg.inv(covariance)

                
            concentration[high_concentration_mask] = (
                (radiancediff_with_bg[:, high_concentration_mask].T @ covariance_inverse @ high_target_spectrum) / 
                (high_target_spectrum.T @ covariance_inverse @ high_target_spectrum)
            )+ adaptive_threshold

            concentration[low_concentration_mask] =(
                (radiancediff_with_bg[:, low_concentration_mask].T @ covariance_inverse @ target_spectrum) / 
                (target_spectrum.T @ covariance_inverse @ target_spectrum)
            )

        new_mean_concentration = np.nanmean(concentration)
        new_std_concentration = np.nanstd(concentration)
        new_adaptive_threshold = new_mean_concentration + new_std_concentration

        if np.abs((new_adaptive_threshold - adaptive_threshold) / adaptive_threshold) > 0.5:
            adaptive_threshold = new_adaptive_threshold
            print("threshold is unstable, keep on iterating")
        else:
            levelon = False
            print("threshold is stable")
    return concentration


### lognornal 匹配滤波算法

In [ ]:
# convert the radiance into log space 整幅图像进行计算
def lognormal_matched_filter(data_cube: np.array, unit_absorption_spectrum: np.array):
    # 获取 以 波段 行数 列数 为顺序的数据
    bands, rows, cols = data_cube.shape
    # 初始化 concentration 数组，大小与卫星数据尺寸一直
    concentration = np.zeros((rows, cols))
    # 对于非空列，取均值作为背景光谱，再乘以单位吸收光谱，得到目标光谱
    log_background_spectrum = np.nanmean(np.log(data_cube), axis=(1,2))
    background_spectrum = np.exp(log_background_spectrum)
    
    # 对当前目标光谱的每一行进行去均值操作，得到调整后的光谱，以此为基础计算协方差矩阵，并获得其逆矩阵
    radiancediff_with_bg = np.log(data_cube) - log_background_spectrum[:, None,None]
    d_covariance = np.log(data_cube)-background_spectrum[:,None,None]
    covariance = np.zeros((bands, bands))
    for row in range(rows): 
        for col in range(cols):
            covariance += np.outer(d_covariance[:, row, col], d_covariance[:, row, col])
    covariance = covariance/(rows*cols)
    covariance_inverse = np.linalg.inv(covariance)

    for row in range(rows):
        for col in range(cols):
            # 基于最优化公式计算每个像素的甲烷浓度增强值
            numerator = (radiancediff_with_bg[:,row,col].T @ covariance_inverse @ unit_absorption_spectrum)
            denominator = (unit_absorption_spectrum.T @ covariance_inverse @ unit_absorption_spectrum)
            concentration[row,col] = numerator/denominator
    return concentration
